## Vectors

In [34]:
x = Int16[10, 5, 3, 11]

# we can access memory directly

x_1 = Base.unsafe_load(pointer(x)) 
# memory is laid out in bytes (8 bits)
x_h = Base.unsafe_load(pointer(x) + 1) 
# Advanced: bits combine half of bits of 4 and bits of 5
# but not as expected since bits are stored "backward" (little-Endian)

sz = sizeof(eltype(x))
x_3 = Base.unsafe_load(pointer(x) + 2sz)

3

In [29]:
Base.unsafe_load(pointer(x) + 4sz) 
# WARNING: May crash. High-performance languages are NOT safe

21504

In [30]:
x

4-element Vector{Int16}:
 10
  5
  3
 11

In [33]:
x = [1.3, 1.2, 1.5]

3-element Vector{Float64}:
 1.3
 1.2
 1.5

## Matrices

In [35]:
A = [1 2 3; 4 5 6; 7 8 9]

3×3 Matrix{Int64}:
 1  2  3
 4  5  6
 7  8  9

In [41]:
sz = sizeof(Int)
Base.unsafe_load(pointer(A) + 3sz)

2

In [42]:
vec(A)

9-element Vector{Int64}:
 1
 4
 7
 2
 5
 8
 3
 6
 9

In [57]:
function mul_rows(A, x)
    m,n = size(A)
    ret =  zeros(m)
    for k = 1:m
        for j = 1:n
            ret[k] += A[k,j] * x[j]
        end
    end
    ret
end

function mul(A, x)
    m,n = size(A)
    ret =  zeros(m)
    for j = 1:n
        for k = 1:m
            ret[k] += A[k,j] * x[j]
        end
    end
    ret
end

mul (generic function with 1 method)

In [79]:
n = 100
A = randn(n,n)
x = randn(n)
mul_rows(A,x) ≈ A*x

true

In [51]:
] add BenchmarkTools

   Resolving package versions...
  No Changes to `~/Documents/Lectures/MATH50003NumericalAnalysis/Project.toml`
  No Changes to `~/Documents/Lectures/MATH50003NumericalAnalysis/Manifest.toml`


In [58]:
using BenchmarkTools

In [62]:
@btime mul_rows(A, x)
@btime mul(A, x)
@btime A*x

  18.228 μs (1 allocation: 896 bytes)
  6.637 μs (1 allocation: 896 bytes)
  1.883 μs (1 allocation: 896 bytes)


100-element Vector{Float64}:
 -11.091082683943975
  15.375023837352812
   1.7863169866347457
   3.7581125078770223
 -11.382201133296679
 -12.391297354798967
  -9.666072728450594
  17.852819140415107
  -6.526323923317621
   2.3809426807502265
 -26.6358042787717
   0.3943165121789224
  -2.6864548720750006
   ⋮
  -8.39162579412428
  12.791452114985063
  -7.941425454384663
  12.516025757522588
 -14.57435214893141
  -7.375001104828355
   4.095507806518102
   4.643869749457856
  -7.70968610579241
   9.519075191195832
  10.605253221605262
  -7.2323433646541035

We can estimate algorithm complexity by considering `n` and `2n` at same time. E.g. if algorithm takes `~Cn^p` operations on `n` then it will take `~2^pCn^p`operations on `2n`, in other words `time on 2n / time on n -> 2^p`

In [80]:
n = 100
A = randn(n,n)
x = randn(n)

B = randn(2n,2n)
y = randn(2n);

In [81]:
(@belapsed mul_rows(B,y))/(@belapsed mul_rows(A,x))

4.411648351648352

In [82]:
(@belapsed mul(B,y))/(@belapsed mul(A,x))

3.8463392588128955

In [83]:
(@belapsed B*y)/(@belapsed A*x)

2.187420284925831